# sklearn IRIS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [1]:
import bentoml
NAME = 'sklearn_iris_clipper'

# load data & train model

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.69 0.   1.16 1.   0.   1.   2.   1.   0.   2.
 0.   1.87 2.   2.   0.   0.   1.   2.   1.   1.28 1.59 1.87 1.   1.
 2.   2.  ]
  mse: 0.086067


In [3]:
%%writefile {NAME}.py

import bentoml
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import ClipperFloatsHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):

    @bentoml.api(ClipperFloatsHandler)
    def predict_clipper(self, inputs):
        outputs = self.artifacts.model.predict(inputs)
        return outputs

Overwriting sklearn_iris_clipper.py


In [4]:
from sklearn_iris_clipper import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

[2020-05-09 03:28:08,303] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-09 03:28:08,681] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+222.g54dcd0e.dirty
creating BentoML-0.4.9+222.g54dcd0e.dirty/BentoML.egg-info
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/artifact
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/bundler
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/cli
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/clipper
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/configuration
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/deployment
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/handlers
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/marshal
creating BentoML-0.4.9+222.g54dcd0e.dirty/bentoml/migrations
creating BentoML-0.4.9+222.g54d

In [5]:
from bentoml.utils import detect_free_port
PORT = detect_free_port()
print(PORT)

53411


In [11]:
#server_url = '/home/ec2-user/bentoml/repository/BentoSvc/20200508043711_5DCCB7'

# Build & Run Bento Service in Docker

In [6]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.clipper import deploy_bentoml
cl = ClipperConnection(DockerContainerManager())
try:
    cl.start_clipper(cache_size=1)
except:
    cl.connect()


20-05-09:03:28:40 WARNING  [clipper_admin.py:141] [default-cluster] Error starting Clipper: Cluster default-cluster cannot be started because it already exists. Please use ClipperConnection.connect() to connect to it.
20-05-09:03:28:40 INFO     [clipper_admin.py:172] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


['bentosvc-predict-clipper:20200508043711-5dccb7']

In [7]:
APP_NAME = saved_path.split('/')[-1].lower()
cl.register_application(APP_NAME, 'floats', 'default_pred', 300000)

20-05-09:03:28:49 INFO     [clipper_admin.py:236] [default-cluster] Application 20200509032808_9ed49b was successfully registered


In [8]:
clipper_model_name, clipper_model_version = deploy_bentoml(
    cl, saved_path, 'predict_clipper',
    build_envs=dict()
)

[2020-05-09 03:28:54,739] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-09 03:28:54,749] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.4.9,  but loading from BentoML version 0.4.9+222.g54dcd0e.dirty
[2020-05-09 03:28:55,428] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image

20-05-09:03:29:48 INFO     [docker_container_manager.py:409] [default-cluster] Found 0 replicas for bentosvc-predict-clipper:20200509032808-9ed49b. Adding 1
20-05-09:03:29:48 INFO     [clipper_admin.py:724] [default-cluster] Successfully registered model bentosvc-predict-clipper:20200509032808-9ed49b
20-05-09:03:29:48 INFO     [clipper_admin.py:642] [default-cluster] Done deploying model bentosvc-predict-clipper:20200509032808-9ed49b.


[2020-05-09 03:29:48,958] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-09 03:29:48,968] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.4.9,  but loading from BentoML version 0.4.9+222.g54dcd0e.dirty


In [11]:
cl.set_num_replicas(clipper_model_name, 1)  # limit workers for benchmark
cl.link_model_to_app(APP_NAME, clipper_model_name)
addr = cl.get_query_addr()
server_url = f"http://{addr}/{APP_NAME}/predict"

20-05-09:03:30:14 INFO     [clipper_admin.py:303] [default-cluster] Model bentosvc-predict-clipper is now linked to application 20200509032808_9ed49b


In [12]:
server_url

'http://localhost:1337/20200509032808_9ed49b/predict'

# Test with requests

In [16]:
import json
import requests

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

headers = {"content-type": "application/json"}
data = json.dumps(
       {"input": X_test[0].tolist()}
)

json_response = requests.post(server_url, data=data, headers=headers)
print(json_response)
print(json_response.text)

<Response [200]>
{"query_id":399371,"output":2.0,"default":false}


# Benchmark

In [17]:
import pandas as pd
import json
import copy
import random


def get_request_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    url = server_url
    method = "POST"
    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=7)
    base_data = X_test[0].tolist()

    def _gen_data():
        raw = copy.deepcopy(base_data)
        raw[0] += random.random() / 10000
        raw[1] += random.random() / 10000
        data = json.dumps({"input": raw})
        return url, method, headers, data

    return _gen_data

get_request_producer()()

('http://localhost:1337/20200509032808_9ed49b/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"input": [5.100026805383531, 1.8000281745233842]}')

In [18]:
from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(), lambda: 1, timeout=10)
b.start_session(60, 900, 600)

======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │     732 │      364 │        0.404865 │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │      nan        │                   │
╘══════════╧═════════╧═══════

# Benchmark with locust

In [17]:
%%writefile benchmark_{NAME}.py
from locust import HttpLocust, TaskSet, task, constant
from functools import lru_cache

import numpy as np
import pandas as pd
import json


@lru_cache(maxsize=1)
def data_producer():

    from sklearn import datasets
    from sklearn.model_selection import train_test_split

    iris = datasets.load_iris()
    x = iris.data[:, 2:]
    y = iris.target

    headers = {"content-type": "application/json"}
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

    def _gen_data():
        raw = X_test[0][0] + np.random.random([2]) / 10000
        data = json.dumps(
               {"input": raw.tolist()}
        )
        return headers, data

    return _gen_data


class WebsiteTasks(TaskSet):

    @task
    def index(self):
        headers, data = data_producer()()
        self.client.post("", data, headers=headers)

class WebsiteUser(HttpLocust):
    task_set = WebsiteTasks
    wait_time = constant(1)

Overwriting benchmark_sklearn_iris_clipper.py


In [18]:
!locust -f benchmark_{NAME}.py --slave & locust -f benchmark_{NAME}.py --slave & locust -f benchmark_{NAME}.py --slave & locust -f benchmark_{NAME}.py -H {server_url} --no-web -t 60s --csv {NAME} -c 1000 -r 99 --master

[2020-03-31 08:39:01,807] ip-172-31-15-25/INFO/locust.main: Starting Locust 0.14.5
[2020-03-31 08:39:01,818] ip-172-31-15-25/INFO/locust.main: Starting Locust 0.14.5
[2020-03-31 08:39:01,865] ip-172-31-15-25/INFO/locust.main: Starting Locust 0.14.5
[2020-03-31 08:39:01,873] ip-172-31-15-25/INFO/root: Waiting for slaves to be ready, 0 of 1 connected
[2020-03-31 08:39:01,998] ip-172-31-15-25/INFO/locust.runners: Client 'ip-172-31-15-25_a2de345b402645a6ac95ea9a0d59ac5d' reported as ready. Currently 1 clients ready to swarm.
[2020-03-31 08:39:02,011] ip-172-31-15-25/INFO/locust.runners: Client 'ip-172-31-15-25_434e81a97e6f456bab73f4d8e392e5e7' reported as ready. Currently 2 clients ready to swarm.
[2020-03-31 08:39:02,051] ip-172-31-15-25/INFO/locust.runners: Client 'ip-172-31-15-25_0fcf1c892827464fad0fad9619610f57' reported as ready. Currently 3 clients ready to swarm.
[2020-03-31 08:39:02,874] ip-172-31-15-25/INFO/locust.runners: Sending hatch jobs of 333 locusts and 33.00 hatch rate to 